## Part 1: Over-representation and Enrichment Analysis

In [ ]:
import pickle
import os
import sys
import matplotlib.pyplot as plt
import urllib
import urllib as ul
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
import networkx as nx
import ast
from prettytable import PrettyTable
import gseapy as gp
from palettable import wesanderson

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(os.getcwd())
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
print(parent_dir)

### 0. Load networks from Section 1

- Gene correlation network
- Patient network from TCGA gene expression data
- Patient network from TCGA DNA methylation data


In [ ]:
intermediate_data_dir = '/data/intermediate/'
raw_data_dir = '/data/raw/'

# Define paths to .gml network files from Section 1
# These paths point to the Gene correlation network, Patient network from (1) TCGA gene expression data and (2) TCGA DNA methylation data.
G_gxp_path = intermediate_data_dir + 'section2_networks_v1/gene_coexpression_network.gml'

# Load the GML graphs into NetworkX graph objects
# nx.read_gml() function reads a graph from a GML file
G_gxp = nx.read_gml(G_gxp_path)  # Gene correlation network

# Get all nodes in each graph
# The nodes represent genes or patients depending on the network
G_gxp_nodes_list = list(G_gxp.nodes())  # Nodes in the gene correlation network

# Define paths to the raw TCGA datasets
# tcga_dnam_path = 'section2_data/ISMB_TCGA_DNAm.pkl'  # TCGA DNA methylation data
tcga_gxp_path = raw_data_dir + 'ISMB_TCGA_GE.pkl'  # TCGA Gene expression data

# Load the gene expression dataset
# pd.read_pickle() function loads a pickled pandas DataFrame or Series
tcga_gxp = pd.read_pickle(tcga_gxp_path)

# For this example, we'll use a CSV file that includes gene symbols
# pd.read_csv() function loads a CSV file into a pandas DataFrame
tcga_gxp_df = pd.read_csv(intermediate_data_dir + 'tcga_ge_df_symbols_t.csv') # Dataset with gene symbols
# Set 'GENES' column as the index for easy access to gene-specific data
tcga_gxp_df.set_index('GENES', inplace=True)

# Extract metadata from the gene expression dataset
# Metadata might include information such as patient IDs, sample conditions, etc.
tcga_gxp_meta = tcga_gxp['datMeta']

# Print the number of nodes in each network
# This provides a quick overview of the size of each network
print(f"Number of nodes in gene correlation network: {len(G_gxp_nodes_list)}")


##### Custom function `draw_network_with_node_attrs()` draws a network with nodes colored and/or shaped based on their attributes. If communities are provided, nodes are colored by their community memberships. A legend is added to indicate the mapping of attributes to colors and shapes.

In [ ]:
from functions import draw_network_with_node_attrs

# Args:
#     G (networkx.Graph): The graph to be drawn.
#     node_attributes (dict): A dictionary where keys are node names and values are dictionaries of attributes.
#     communities (List[List[Any]], optional): A list where each sublist contains the nodes belonging to a community. Default is None.
#     title (str, optional): The title of the plot. Default is 'Network Visualization'.
#     color_attr (str, optional): Node attribute to color nodes by. Default is None.
#     shape_attr (str, optional): Node attribute to shape nodes by. Default is None.
#     figsize (tuple, optional): The size of the figure. Default is (20, 10).
#     layout (str, optional): The layout algorithm for positioning nodes ('spring', 'circular', etc.). Default is 'spring'.
#     cmap_name (str, optional): The name of the colormap to use for coloring. Default is 'tab20'.
#     with_labels (bool, optional): Whether to draw labels for the nodes. Default is False.

In [ ]:
# Display the shape of the dataframe
print("\nShape of the dataframe 'tcga_gxp_df' (rows, columns):")
### YOUR CODE HERE ###
print()

# List the columns in the dataframe
print("\nList of columns in the dataframe 'tcga_gxp_df':")
### YOUR CODE HERE ###
print()

##### We will use [cancer gene calatogue (CGC)](https://cancer.sanger.ac.uk/census) from the Catalogue Of Somatic Mutations In Cancer (COSMIC) to annotate genes in our network. 

- `'Tier'`
    - To be classified into Tier 1, a gene must possess a documented activity relevant to cancer, along with evidence of mutations in cancer which change the activity of the gene product in a way that promotes oncogenic transformation.
    - Tier 2 consists of genes with strong indications of a role in cancer but with less extensive available evidence.
- `'Hallmark'`
    - New overviews of cancer gene function focused on hallmarks of cancer pull together manually curated information on the function of proteins coded by cancer genes and summarise the data in simple graphical form. They present a condensed overview of most relevant facts with quick access to the literature source, and define whether a gene has a stimulating or suppressive effect via individual cancer hallmarks.

In [ ]:
cancer_genes_path = 'section2_data/Census_allFri Jul  5 14_32_40 2024.csv'
cancer_genes_df = pd.read_csv(cancer_genes_path)
print(cancer_genes_df.columns)
cancer_genes_df.head(2)

In [ ]:
cancer_genes_df = cancer_genes_df[cancer_genes_df['Tier']==1]
cancer_genes = cancer_genes_df['Gene Symbol'].tolist()

# Create the dictionary with node labels as keys and boolean as values
# Complete the dictionary comprehension to check if each node is in the list of cancer genes

### YOUR CODE HERE ###
found_in_cancer_genes = {node: bool(node in cancer_genes) }

node_attributes={}
node_attributes['cancer_gene'] = found_in_cancer_genes

In [ ]:
node_attributes

### 1. Gene Network Analysis with `G_exp`

We are going to use `'KEGG_2021_Human'` as the gene set. KEGG (Kyoto Encyclopedia of Genes and Genomes) is a knowledge base for systematic analysis of gene functions, linking genomic information with higher order functional information [(Kanehisa et al., 2000)](https://doi.org/10.1093/nar/28.1.27).

In [ ]:
# Alternative gene sets like 'MSigDB_Hallmark_2020' can also be used.
gene_sets = 'KEGG_2021_Human'

In [ ]:
# You can also retrieve and display the list of available gene sets
gene_set_list = gp.get_library_name()
print(gene_set_list)

#### 1.1 Over Representation Analysis (ORA)

Over-representation analysis (ORA) is a method used to identify which predefined gene sets are disproportionately represented in a given set of genes compared to what would be expected by random chance [(Huang et al., 2009)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2615629/). We recommend using Over-representation analysis (ORA) only when Gene Set Enrichment Analysis (GSEA) is not suitable. Although we are using the [`gseapy`](https://doi.org/10.1093/bioinformatics/btac757) library for ORA in this the tutorial, it's important to note that ORA and GSEA are distinct methods.

##### 1.1.1 ORA on gene correlation network

In [ ]:
# Perform ORA on all nodes in the gene correlation network

### YOUR CODE HERE ###
enr_all_nodes = gp.enrichr()

In [ ]:
# Display the top 10 enrichment results

### YOUR CODE HERE ###

##### 1.1.1 ORA on gene clusters

Clustering on Gene Correlation Network

- We use community detection algorithm to identify communities in the network: 
    - The `greedy_modularity_communities()` function in `networkx` implements a community detection algorithm that optimises modularity using a greedy approach. It iteratively merges pairs of nodes or communities that result in the largest increase in modularity until no further improvement is possible. Modularity measures the density of links inside communities compared to links between communities, aiming to maximise this value to identify densely connected groups within the network.

In [ ]:
# Cluster the gene correlation network using the greedy modularity communities algorithm
communities = nx.algorithms.community.modularity_max.greedy_modularity_communities(G_gxp)

# Print the number of detected communities

### YOUR CODE HERE ###
print(f'The network has {} communities.\n')

In [ ]:
# Create subgraphs for each community
subgraphs = []
for community in communities:
    
    subgraphs.append(G_gxp.subgraph(community))

# Print the number of nodes in each subgraph
# And draw the subgraph
for i, subgraph in enumerate(subgraphs):
    print(f'Community {i+1} has {subgraph.number_of_nodes()} nodes.')
    
    ### YOUR CODE HERE ###
    
    plt.show() # Forces the plot to be displayed
    
    

#### Visualising the Communities

In [ ]:
# Visualise the gene correlation network with community memberships

### YOUR CODE HERE ###
draw_network_with_node_attrs() 

#### Perform ORA on Each Community

Performing ORA on individual clusters can help in understanding distinct biological significance of each cluster, revealing how certain pathways or functions are associated with specific subsets of genes.

In [ ]:
# Function to perform ORA on a given list of genes
def communityORA(genes):
    enr = gp.enrichr(gene_list=genes, gene_sets=[gene_sets], organism='human', outdir=None)
    return enr

# 1. Convert communities to lists for ORA 2. Sort the list of communities by their length in descending order

### YOUR CODE HERE ###
communities = 
communities = 

# Perform ORA for three selected communities
community1_enr = communityORA(communities[0])
community2_enr = communityORA(communities[1])
community3_enr = communityORA(communities[2])

# Print the top 10 results for each community
x = PrettyTable()
x.field_names = ["Community 1", "Community 2", "Community 3"]
for i in range(10):
    x.add_row([community1_enr.results['Term'][i], community2_enr.results['Term'][i], community3_enr.results['Term'][i]])
print(x)


#### 1.2 Gene Set Enrichment Analysis (GSEA)

Gene Set Enrichment Analysis (GSEA) is a genome-wide expression analysis method designed to interpret expression profiles focusing on pre-defined gene sets [(Subramanian et al., 2005)](https://doi.org/10.1073/pnas.0506580102). These gene sets are curated based on prior biological knowledge, such as published information about biochemical pathways or patterns of coexpression observed in previous experimental studies. The genes can be ordered in a ranked list, according to their differential expression between the classes. The primary objective of GSEA is to assess whether the genes within a given gene set tend to occur toward the top (or bottom) of the ranked list. This ranking is based on the correlation between gene expression and a particular phenotypic class distinction. By evaluating the distribution of gene set members within the ranked list, GSEA identifies whether the set is correlated with the phenotypic class, thus providing insights into underlying biological mechanisms. This method contrasts with traditional single-gene analysis by focusing on the collective behavior of gene sets, thereby uncovering biologically significant patterns that might be overlooked when examining individual genes in isolation. We use `gseapy` to perform GSEA with the gene set `KEGG_2021_Human`.

<!-- (https://pnnl-comp-mass-spec.github.io/proteomics-data-analysis-tutorial/gsea.html) for referencing.  -->


In [ ]:
# Display the first five rows of the dataframe "tcga_gxp_df"
tcga_gxp_df.head(5)

In [ ]:
tcga_gxp_meta.head(3)

#### Assign classes based on phenotypic attributes in metadata (e.g., smoking status)

In [ ]:
# Create a dictionary to store the class assignments for each sample
classes = {}

# Iterate through the columns (samples)
for sample in tcga_gxp_df.loc[G_gxp_nodes_list].columns: # ".loc[G_gxp_nodes_list]" grabs rows of specified gene symbols
    
    # Assign class labels based on the smoking status from the metadata
    
    if tcga_gxp_meta.loc[sample, 'Smoked'] == 'Smoker':
        ### YOUR CODE HERE ###
    elif tcga_gxp_meta.loc[sample, 'Smoked'] == 'Never':
        ### YOUR CODE HERE ###
    else:
        pass


In [ ]:
classes

In [ ]:
tcga_gxp_df.loc[G_gxp_nodes_list].head(2)

In [ ]:
# Perform GSEA using the prepared data and class assignments
gs_res = gp.gsea(data=tcga_gxp_df.loc[G_gxp_nodes_list], gene_sets=gene_sets, cls=classes, permutation_num=100, outdir=None, method='signal_to_noise', threads=4, seed=7)

# Display the top results from the GSEA

### YOUR CODE HERE ###

#### Visualising GSEA Results

Once you have performed GSEA, the next step is to visualise the results. Visualisation helps in interpreting the biological roles of the enriched gene sets. Here, we visualise GSEA results with Barcode Enrichment Plot, Heatmap, Clustermap, and Dot Plot. 

### Barcode Enrichment Plot

Barcode Enrichment Plot shows the positions of members of a given gene set in a ranked list of enrichment scores for the top enriched terms. The scores are ranked left to right from smallest to largest. The ranked scores are represented by a shaded bar, forming a pattern like a barcode. 

In [ ]:
# Extract the enriched terms from the GSEA results. The terms represent pathways or functional categories that are significantly enriched in the dataset.

### YOUR CODE HERE ###
terms = 

# Plot the top 5 enriched terms
# The plot function visualizes the enrichment results
axs = gs_res.plot(terms[:5], show_ranking=False, legend_kws={'loc': (1.05, 0)})

You can view and extract leading-edge genes from GSEA results. Leading-edge genes are the subset of genes that contribute most to the enrichment score.

In [ ]:
# View leading-edge genes from the GSEA results. 
gs_res.res2d[['Term', 'Lead_genes']].head(10)

### Heatmap Visualisation

`gseapy` provides a heatmap function to visualise the expression levels of the leading-edge genes. The heatmap provides a visual representation of how these genes are expressed across different samples in relation to their assigned phenotypic classes. 

In [ ]:
# Import the heatmap function from gseapy
from gseapy import heatmap

# Select the index of the term to visualize
i = 0

# Extract the genes contributing to the enrichment of the selected term

### YOUR CODE HERE ###
genes = 
print(genes)

In [ ]:
# Generate a heatmap of the expression levels of the leading-edge genes
ax = heatmap(df=gs_res.heatmat.loc[genes], z_score=0, title=terms[i], figsize=(15, 4))

# Update the x-tick labels with the class labels
xtick_labels = [classes[item.get_text()] for item in ax.get_xticklabels()]
ax.set_xticklabels(xtick_labels, rotation=45, ha='right')
ax.plot()

### Clustermap Visualisation

The function `clustermap` from `seaborn` is used to create a clustered heatmap. It not only shows the expression levels of the leading-edge genes but also clusters them based on similarity, providing additional insights into gene expression patterns. The cluster map includes dendrograms, which show the hierarchical clustering of both genes and samples, helping to identify groups of co-expressed genes and similar samples.

In [ ]:

# Import the clustermap function from seaborn
from seaborn import clustermap

# Select the index of the term to visualize
i = 2

# Extract the genes contributing to the enrichment of the selected term
genes = gs_res.res2d.Lead_genes[i].split(";")

# Extract the relevant subset of the heatmap data

### YOUR CODE HERE ###
data = 

# Rename the columns based on the class assignments
data.rename(columns=classes, inplace=True)

# Generate a cluster map of the expression levels of the leading-edge genes
ax = clustermap(
    data=data,  # The data to cluster
    method='average',  # Clustering method
    metric='euclidean',  # Distance metric
    z_score=0,  # Standardize the data along the rows
    figsize=(14, 4),  # Size of the figure
    dendrogram_ratio=0.2,  # Ratio of the dendrogram
    colors_ratio=0.03,  # Ratio of the colors
    cbar_pos=(0.02, 0.1, 0.05, 0.1)  # Position of the color bar
)

### Dot Plot Visualisation

Use the `dotplot` function in `gseapy` to create a visual representation of the GSEA results. Here we use "FDR q-val" to determine the dot sizes, which represents the false discovery rate adjusted p-values. We display normalised enrichment score (NES) value as the x-axis. 

In [ ]:
# Import the dotplot function from gseapy
from gseapy import dotplot

# Generate the dot plot for the GSEA results
# The dotplot function visualizes the enrichment results, focusing on the FDR q-values
ax = dotplot(gs_res.res2d,
             column="FDR q-val",  # Column to be used for dot size
             title=gene_sets[0],  # Title the plot as the chosen gene set name 
             cmap=plt.cm.viridis,  # Color map for the dots
             size=5,  # Size of the dots
             figsize=(4, 5),  # Size of the figure
             cutoff=1)  # Cutoff for displaying the terms

### GSEA on Clusters

Similarly to ORA, GSEA can also be performed on individual communities after clustering. This allows for a more granular analysis, revealing pathways and functions that are enriched within particular subgroups of the data.


In [ ]:
# Define a function to perform GSEA on a given list of genes (community)
def communityGSEA(genes, gene_sets='KEGG_2021_Human', classes=classes):
    gs_res = gp.gsea(data=tcga_gxp_df.loc[genes], gene_sets=gene_sets, cls=classes, permutation_num=100, outdir=None, method='signal_to_noise', threads=4, seed=7, min_size=1) 
    return gs_res

# Perform GSEA for three community

### YOUR CODE HERE ###
community1_gsea = communityGSEA()
community2_gsea = communityGSEA() 
community3_gsea = communityGSEA()


In [ ]:
# Display the top results for the first community

### YOUR CODE HERE ###

In [ ]:
# Extract the enriched terms from the GSEA results for the first community
terms = community1_gsea.res2d.Term

# Plot the top 5 enriched terms for the first community
axs = community1_gsea.plot(terms[:5], show_ranking=False, legend_kws={'loc': (1.05, 0)})